# 환경설정

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
WAIT = 3
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

# 함수 정의

In [2]:
# css 찾을때 까지 10초대기
def time_wait(num, code,driver):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [3]:
def find_search_input(key_word,driver):
    # 검색창 찾기
    driver.switch_to.default_content()
    time_wait(2, 'div.input_box > input.input_search',driver)
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    driver.implicitly_wait(3)
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    sleep(5)
    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
    sleep(1)

In [4]:
# frame 변경 메소드
def switch_frame(frame,driver):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    driver.implicitly_wait(1)


In [5]:
# 페이지 다운
def page_down(num,driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [6]:
# 페이지 다운
def page_end(driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    body.send_keys(Keys.END)

In [7]:
def find_review_btn(driver):
    sleep(2)
    try:
        menu_list = driver.find_elements(By.CSS_SELECTOR,'a.tpj9w')
        if len(menu_list) ==7:
            return menu_list[-1]
        else:
            return menu_list[-2]
    except:
        print('리뷰가 존재하지 않습니다.')

In [8]:
def crawling_menu_review(driver):
    #-----리뷰 버튼 클릭-------
    find_review_btn(driver).click()
    SCROLL_PAUSE_SEC = 1
    sleep(1)
    page_down(1,driver)
    driver.implicitly_wait(WAIT)
    #----------------메뉴, 크롤링 블록----------------
    menu_list = list()
    review_list = list()
    try:
        # --------------메뉴 크롤링 블록--------------------------
        try :
            #메뉴 요소 찾기
            menu_elements  = driver.find_elements(By.CSS_SELECTOR,'div.place_fixed_subtab > div > div:nth-child(1) > div > div > div > span > a > span:first-child')
            sleep(SCROLL_PAUSE_SEC)
            driver.implicitly_wait(WAIT)
            try:
                #왼쪽 버튼 찾기
                left_btn = driver.find_element(By.CSS_SELECTOR,'a.PznE8.aNw43')
                driver.implicitly_wait(WAIT)
                left_btn.click()
                driver.implicitly_wait(WAIT)
            except:
                pass

            try:
                #오른족 버튼 찾기
                right_btn = driver.find_element(By.CSS_SELECTOR, 'div.place_fixed_subtab > div > div:nth-child(1) > div > div > a.ZCqf_.aNw43')
                driver.implicitly_wait(WAIT)
            except:
                pass
            #-----------여기까지-----------
            i = 0
            # menu_elentents가 존재하지 않는다면 except로 갈 것이다.
            try: 
                while True:
                    if menu_elements[i].text != '': #요소가 존재하고 메뉴 텍스트도 보일때
                        menu_list.append(menu_elements[i].text)
                        if i == len(menu_elements)-1: # 메뉴 리스트 길이면 나온나
                            break
                        i+=1
                    else:
                        right_btn.click()
                        sleep(SCROLL_PAUSE_SEC)
                        driver.implicitly_wait(WAIT)
            except:
                pass

            #last_scroll = driver.execute_script("return document.body.scrollHeight")
            #view_more_count = 0

        except:
            pass

        # --------------리뷰 크롤링 블록--------------------------
        try:
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(1)
                driver.implicitly_wait(SCROLL_PAUSE_SEC)

                #if view_more_count ==19:
                    #break
                try :
                    #더보기 버튼 클릭
                    driver.implicitly_wait(WAIT)
                    driver.find_element(By.CSS_SELECTOR,'a.fvwqf').click()
                    #view_more_count += 1

                except:
                    #더보기 버튼이 없다면 while문을 빠져나온다.
                    break
                    # 끝까지 스크롤 다운

                #new_scroll = driver.execute_script("return document.body.scrollHeight")
                #driver.implicitly_wait(WAIT)
                #if last_scroll == new_scroll:
                #    break
                #else:
                #    last_scroll= new_scroll

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            #가게 리스트 마다 방문자 리뷰 가져오기
            review_elements = driver.find_elements(By.CSS_SELECTOR,"span.zPfVt")
            for i in review_elements:
                review_list.append(i.text)
        except:
            pass
    
    except:
        pass
    
    return menu_list, review_list

# 검색 지역 횟집의 리뷰를 크롤링.
## 한번에 하는 코드 / refresh()를 해도 응답이 없음

In [31]:
naver_인천_횟집_review_dic_page1['연안부두 목포신안18호횟집']['review']

['대한민국이가 왔던집이라네요..\n새우튀김은 언제나 맛있어요.',
 '갈치구이 백반 2인상이에요.\n맛있어요.',
 '새우오징어튀김맛있어요',
 'B코스로 2인 12만원으로 먹었어요 회랑 조개찜이랑 대하구이랑 각종해산물들에 마지막에 칼국수까지 다 양하게 맛 보고 왔어요~ 보통 회 먹으면 매운...',
 '그냥',
 '친절하고 생선구이 맛있어요',
 '.....',
 '좋아요',
 '좋아요',
 '좋아요',
 '좋네요',
 '맛있게 먹었어요\n매운탕 대신 칼국수 먹었는데, 좋았어요',
 '밥도 쉬냄가 났고,밑반찬도 별루고 비싸기만해요',
 '회신선하게 잘먹엇어요',
 '생선구이백반 3인 시키면 생선 5종류 나옴 반찬은 미\n역국, 어묵볶음, 김치, 톳? 정도 나옴 새우튀김 10.0 \n삼둥이 슈돌에 나온 곳이라고 함 ㅋㅋ',
 '맛있네요.',
 '굿',
 '현지에가서 특산물 먹으면 바가지에다 먹을꺼없다더니 진짜 밥값 아까웠습니다.제발 먹는장사 그렇게 하지마세요',
 '남이 입으로 먹던 튀김이 다시 튀김기름에 들어 갔다 나와도 괜찮은가요? 튀김을 시켜 먹었는데 튀김 옷 이 다 익지 않아 텁텁한 밀가루 냄새가 났어요 ...',
 '좋아요',
 '보통',
 '맛있어요',
 '친절해요~',
 '굳',
 '굿',
 '굿',
 '생선구이가 맛있어요',
 '친절해요',
 '굿',
 '좋아요',
 '그냥',
 '정말 웬만하면 이런 글 안쓰는데 다른 분들도 아셔야 할 것 같아서 남깁니다 호객행위 하셔서 믿고 한 번 들어가서 먹어보자해서 주문했는데 가격에 비...',
 '별루 뭐 딱히 흠 옆에분은 반찬 본인이가서 담아왓 다고 종업원분 한소리들으셔서 화가나셨는지 종업 원분테 계속뭐라하시던데.. 저도 제가가서 담...',
 '칼국수는 별로였어요 밑반찬 잘나온다고 해서 들어갔는데 아이가 먹을건 생선구이만 있습니다',
 '좋아요',
 '별로입니다. 가격에 비해 양이 너무 적고요. 2층에 사 람을 올려놓고 벨도 없이 서빙하시는 분도 없네요. 그리고 회에 대한 설명도 하나 없는 이곳에 다

In [9]:
def crawling(driver, i, store_elements,review_dic):
    driver.implicitly_wait(1)
    store_elements[i].click() #li의 스토어를 클릭
    driver.implicitly_wait(WAIT) # 2초 휴식
    try:
        #다른 iframe으로 변경
        switch_frame('entryIframe',driver)
        time_wait(2, '.Fc1rA',driver)

        # -----매장명 가져오기-----
        store_name = driver.find_element(By.CSS_SELECTOR,'.Fc1rA').text
        driver.implicitly_wait(2)
        review_dic[store_name] = {}
        
        # -----평점 가져오기-----
        try :
            rank = driver.find_element(By.CSS_SELECTOR,'div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot.LXIwF > em')
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['rank'] = rank.text
        except:
            pass

        # -----위치 가져오기-----
        try : 
            location = driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['location'] = location
        except:
            pass
        
        
        #---------------문제--------------
        memu_list, review_list = crawling_menu_review(driver)
        
        # 메뉴 크롤링
        review_dic[store_name]['menu'] = menu_list
        
        # 방문자 리뷰 크롤링
        review_dic[store_name]['review'] = review_list
        driver.implicitly_wait(WAIT)
        
    except:
        pass
    
    memu_list, review_list = crawling_menu_review(driver)
        
    # 메뉴 크롤링
    review_dic[store_name]['menu'] = memu_list

    # 방문자 리뷰 크롤링
    review_dic[store_name]['review'] = review_list
    driver.implicitly_wait(WAIT)
    

## 6번에 걸쳐 1페이지를 크롤링

In [10]:
def review_crawling(key_word, page_num):
    review_dic = dict()
    page_num = page_num-1
    for idx in range(6):
        url = 'https://map.naver.com/v5/search'
        driver = webdriver.Chrome('./chromedriver')
        driver.get(url)
        driver.implicitly_wait(2)
        
        # 검색어 입력
        find_search_input(key_word,driver=driver)
        driver.implicitly_wait(2)
        
        #프레임 변경
        switch_frame('searchIframe',driver)
        driver.implicitly_wait(WAIT)
        
        # 페이지 버튼 찾기
        pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
        if page_num <= 5:
            pages[page_num].click()
            driver.implicitly_wait(WAIT)
        else:
            driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
            driver.implicitly_wait(WAIT)
            driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
            driver.implicitly_wait(WAIT)
            
        driver.implicitly_wait(WAIT)
        page_down(40,driver)
        sleep(1)
        driver.implicitly_wait(1)
        page_down(5,driver)
        sleep(2)
        driver.execute_script("window.scrollTo(0, 0);")
        driver.implicitly_wait(WAIT)
        store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
        print('total store_list : ', len(store_elements))
        
        
        #체크포인트
        ch_pt1 = len(store_elements) // 6
        ch_pt2 = ch_pt1*2
        ch_pt3 = ch_pt1*3
        ch_pt4 = ch_pt1*4
        ch_pt5 = ch_pt1*5
        
        if idx == 0:
            for i in range(ch_pt1):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == ch_pt1-1:
                    print('first crawling store_list', ch_pt1)
                            
        elif idx == 1:
            for i in range(ch_pt1,ch_pt2):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == ch_pt2-1:
                    print('second crawling store_list', ch_pt2-ch_pt1)
                    
        elif idx == 2:
            for i in range(ch_pt2,ch_pt3):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == ch_pt3-1:
                    print('third crawling store_list', ch_pt3-ch_pt2)
                    
        elif idx == 3:
            for i in range(ch_pt3,ch_pt4):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == ch_pt4-1:
                    print('fourth crawling store_list', ch_pt4-ch_pt3)
                    
        elif idx == 4:
            for i in range(ch_pt4,ch_pt5):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == ch_pt5-1:
                    print('fiveth crawling store_list', ch_pt5-ch_pt4)
            
        else :
            for i in range(ch_pt5,len(store_elements)):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == len(store_elements)-1:
                    print('sixth crawling store_list', len(store_elements)-ch_pt5)
                

    # json 파일로 저장
    with open(f'naver_{key_word}_review_dic_page{page_num+1}.json', 'w', encoding='utf-8') as f:
        json.dump(review_dic, f, indent=4, ensure_ascii=False)
    
            
        
        
    
    
    

In [11]:
key_word_list = ['서울 횟집', '인천 횟집', '부산 횟집', '대구 횟집', '대전 횟집', '울산 횟집', '광주 횟집']

In [16]:
review_crawling(key_word_list[1],1)

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_29948\4000499933.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  53
first crawling store_list 8
total store_list :  53
second crawling store_list 8
total store_list :  53
third crawling store_list 8
total store_list :  53
fourth crawling store_list 8
total store_list :  53
fiveth crawling store_list 8
total store_list :  53
sixth crawling store_list 13


In [18]:
with open('naver_인천 횟집_review_dic_page1.json',encoding='utf-8') as f:
    naver_인천_횟집_review_dic_page1 = json.load(f)
print(len(naver_인천_횟집_review_dic_page1))
naver_인천_횟집_review_dic_page1.keys()

46


dict_keys(['대신수산 영종구읍뱃터점', '진복호', '내고향강원도', '보광호', '연안부두 목포신안18호횟집', '충청도7호', '먹고보자 호남횟집', '자연도횟집', '보길도횟집', '사도시 인천구월점', '덜렁이네 횟집', '어다리 부평점', '서해안회타운', '삼복호', '갯벌횟집', '큰바다회수산', '송도어신', '전라도똑소리', '월미도부산태종대회집', '궁참치', '맘모스회센터28호', '어쭈구리 대박났네', '삐삐부인진동왔네', '서해촌', '석모도회센터', '동해횟집', '나는바다다', '바다사나이', '청기와횟집', '청라횟집명가', '가덕도횟집', '막줄래횟집', '인천항구횟집', '허벌나게많이드립니다', '통영수산물회센터', '어부신광호', '어부횟집', '동해회조개구이', '거잠포횟집', '남동어시장', '회마루', '대우횟집', '전남회조개구이', '청라광어랑우럭이랑', '민영바다식당', '연안부두 횟집 호남6호이모네'])

In [40]:
with open('naver_울산 횟집_review_dic_page1.json',encoding='utf-8') as f:
    naver_울산_횟집_review_dic_page1 = json.load(f)
    
with open('naver_서울 횟집_review_dic_page1.json',encoding='utf-8') as f:
    naver_서울_횟집_review_dic_page1 = json.load(f)

In [42]:
len(naver_서울_횟집_review_dic_page1)

50

In [47]:
review_crawling(key_word_list[2],1)

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_21700\4000499933.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  54
first crawling store_list 9
total store_list :  54
second crawling store_list 9
total store_list :  54
third crawling store_list 9
total store_list :  54
fourth crawling store_list 9
total store_list :  54
fiveth crawling store_list 9
total store_list :  54
sixth crawling store_list 9


In [33]:
review_crawling('인천 횟집', 1)

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_19348\4000499933.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  53
first crawling store_list 8
total store_list :  53
리뷰가 존재하지 않습니다.


AttributeError: 'NoneType' object has no attribute 'click'

In [24]:
with open('naver_인천 횟집_review_dic_page1.json',encoding='utf-8') as f:
    naver_인천_횟집_review_dic_page1 = json.load(f)
naver_인천_횟집_review_dic_page1

{'내고향강원도': {'rank': '4.5',
  'location': '인천 중구 연안부두로75번길 27-2 맞은편, 네비에 내고향강원도를 검색하시면 정확합니다.',
  'menu': ['매운탕',
   '광어',
   '물회',
   '우럭',
   '회정식',
   '멍게',
   '라면',
   '초밥',
   '오징어',
   '광어회',
   '새우',
   '새우구이',
   '알밥',
   '회초밥'],
  'review': ['예전보다 가격 많이 올랐지만 맛있어요',
   '통통새우구이 너무 맛나지만 양이 적어서 순식간에 다 먹었어요..ㅜㅜ',
   '맛있어요~ 스끼다시도 잘나와요~~',
   '회 내오는 접시에 돌 등을 깔아서 좀 더 시원하게 먹을 수있었으면 좋겠어요.',
   '회랑 스끼다시 맛있어요 또갈꺼에요 가성비 좋아요 아이들도 엄청 잘먹었어요 매운탕에 라면사리까지...감동이예요',
   '늘 사람이 많아서 바쁘게 움직이세요.\n최근에는 매운탕도 5천원씩으로 가격을 받습니다.',
   '광어회 너무 탱탱하고 맛있어요~',
   '배달을 시켜서 매장 관련해서 모르지만 맛은 있습니다.\n아쉬운점은 양이 생각보다 적었습니다.',
   '가족모임 좋은시간\n맛있게 먹었어요^^~~',
   '음식이 신선하고 먹을꺼리들이 푸짐하고 많아서 조\n아요~리필도 해주시고 늘 맛있게 잘 먹고 옵니다 (초\n밥이랑회는 먹다가 찍었어요깜빡)',
   '너무불친절해요.... 벨눌러도 대답한마디없이 서계 시고 밥주문했ㄷㅓ니 밥이 아직안되서요도아니라 없 는데요?이럼서... 식구중에 생일자가있어서 갔...',
   '가격이 저렴하면서 싱싱한회 맛잇어요',
   '스페셜 코스 좋아요',
   '통영수산 오랫만에 갔는데 매운탕추가비용이 생겼\n어요₩5000원 전엔 그냥주셨는데. 조금아쉬웠어요. \n그래도 먹을게 많아서 맛있고 조아요 잘 먹었...',
   '횟집',
   '좋아요',
   '굿',
   '굿',
   '좋아요',
   '굿',
   '굳',
   '굿

In [25]:
len(naver_인천_횟집_review_dic_page1)

44

In [29]:
naver_인천_횟집_review_dic_page1['전남회조개구이']

{'rank': '4.59',
 'location': '인천 중구 연안부두로75번길 27-2 호남회센타 6호',
 'menu': ['매운탕', '낙지', '물회', '홍합탕', '광어'],
 'review': ['너무 푸짐해서 배터지게 먹고 왔어요~ 랍스타 대박~! !! 지난번에 다시 들릴거라고 사장님과 약속했는데 잊지않고 반갑게 알아봐 주셔서 너무 좋았어...',
  '60첩 반상 유명하다해서 해산물 회 좋아하는 친정부 모님 모시고 다녀왔어요~ 엄마 아빠가 너무 푸짐하 고 맛있다고~오랜만에 싱싱한 해산물 회드셨...',
  '처음으로 방문한 이곳~ 너무 배고파서 허겁지겁 먹 고나서야 사진찍었네ㅋㅋㅋㅋ 이모님??기억하시죠 ? 몇번이나 더달라해도 인상한번 안쓰시고 인...',
  '올만에 가서 그런지 더더욱 맛있었고 날씨가 더워서 그런지 시원한 물회가 아주 기억에 남았네여~ 조만간 또 들리겠습니당ㅎㅎ',
  '저번 방문에 괜찮아서 초복 겸 시부모님 맛있는거 대 접해 드리고싶어 모시고 갔더니 역시나 너무 좋아하 시네요~ 저번에 왔었다고 사장님께 말씀 드렸...',
  '너무 맛있어서 식구들 꼬셔서 3번째 방문~ 이러다 저 희집 거덜 날꺼 같아여~ㅎㅎ 항상 친절하게 대해주 시고 신선한 음식 해주셔서 기분좋게 다녀오...',
  '처음 갔는데 너무 친절하고 다~~맛있어요~! 특히 매 운탕이 너~~무 맛있어요~! 사장님께 다음에 다시 들 린다고 약속하고 나왔는데~꼭! 지킬 겁니다~ ...',
  '기본적으로 회도 싱싱하고 사장님도 친절하고 너무 즐겁게 먹고왔네요!! 회랑 스끼다시 제대로 먹고싶으 면 요기 만한데 없는거 같아요~ 다음에는 부모...',
  '먹을만해요~~ 연안부두 60첩 1인 4만원이라는 가격은 미끼일뿐 2명이서 가면 비싸고 4만원에 60첩이라는건 4인이상 일때 말하는거!!!',
  '끝인줄 알았는데 종류 다양하게 계속 나오네요~^-^ 음식도 맛있고 사장님도 친절하시고아이들하고 즐거운 점심 먹고왔습니다~',
  '맛난회 먹고싶어 추천받아 갔는데 기본반찬부

In [48]:
with open('naver_부산 횟집_review_dic_page1.json',encoding='utf-8') as f:
    naver_부산_횟집_review_dic_page1 = json.load(f)
naver_부산_횟집_review_dic_page1

{'광안리 산해횟집': {'rank': '4.51',
  'location': '부산 수영구 광안해변로344번길 17-20 풍경타워 8층 광안리 산해횟집',
  'menu': ['조개구이',
   '장어',
   '장어구이',
   '조개',
   '매운탕',
   '모듬회',
   '새우',
   '가리비',
   '치즈',
   '라면',
   '맥주',
   '새우구이',
   '누룽지',
   '전복',
   '우럭'],
  'review': ['청사포 뷰좋고~ 맛있는 횟집!! 발견~~ 싱싱한 장어\n구이 최고~ 사장님 이모님 친절하셔서~~ 다 먹고..기\n분까지 좋게 왔답니다~ 다음번에는 조개구이...',
   '자리에 앉아서 바다가 보여서 분위기가 너무 좋아서 회도 달고 맛있었어요 ㅎㅎ',
   '음..',
   '또왔는데 역시나 뷰도 좋고 여전히 맛있네요 ~ 직원분들도 친절하셔서 편하게 맛있게 잘 먹고왔습니다 : )',
   '첫 월급 기념으로 부모님 모시고 갔습니다 ㅎㅎ 조 개구이는 역시 바닷가에서 먹으니 제맛이네요,, 회도 신선하니 맛있었고 이모님이 구이 종류는 다 ...',
   '오랜만에 낮에 일찍 방문해서 회한접시 먹고왔습니다!! 여전히 음식도 맛있고 뷰는 말할것도 없네요!!',
   '👍',
   '히파리 하시는 아주머니가 친절해서 갔는데 엉망입 니다 쯔게다시 부족한건 말할것도 없구요 회도 신선 하지 않고 매운탕은 조미료맛 강하고 반찬도 ...',
   '쉬는날 부모님들 모시고 오랜만에 방문해서 조개 먹 었습니다!!! 정신없이 먹느라 사진도 하나 못건졌지 만 정말 맛있게 먹고 왔어요ㅎㅎㅎ 갈때마다 ...',
   '제가 이곳저곳 돌아다니면서 많은 가게를 가봤지만 여기 가게만큼 히파리치는 이모님이 불편을 주는 가 게는 없었습니다 관광만 하러 청사포에 갔는...',
   '추석연휴 잘보내고 바다보러 청사포로갔다 착한횟집 또다시 방문~ 싱싱한 장어구이 맛나게먹고 왔네요 바람도 선선하니 불고 배도부르니 기분짱입니...',
   '저녁에 가서 전

In [49]:
naver_서울_횟집_review_dic_page1

{'목포명가 세곡점': {'rank': '4.28',
  'location': '서울 마포구 마포대로 109',
  'menu': ['참치', '회덮밥', '참치회', '초밥', '참치비빔밥', '비빔밥', '알탕'],
  'review': ['점심시간에 엄청붐벼서인지 여자 종업원분이 좀 불\n친절하시네요.ㅠ 음식은 무난, 평범합니다. 런치코\n스 먹었는데 참치 품질과 맛이 쏘쏘였어요.',
   '사장님도 직원분도 친절하시고 참치신선하고 대접할 때 좋은 곳입니다.',
   '좋아요',
   '재방문 의사 없음',
   '회덮밥 맛있어요^^',
   '참치회가 덜 해동돼서 나왔어요 ;; 쏘쏘..',
   '굿여',
   '평범합니다',
   '아주 맛나네요',
   '맛있네요',
   '음… 재방문의사 없습니다',
   '여태 먹었던 참치집 중 손에 꼽는 맛집! 신선해요!',
   '좋아요',
   '굳',
   '참치 넘 맛나요~ 참치 비빔밥은 지금도 군침이 고일\n정도로 맛있구요. 써빙 이모님도 친절하시구~ㅎ 또 \n가려구요 ^^*',
   '여름이라 시원하게 메밀이랑 초밥 맛있게 먹었습니다 초밥이랑 궁합도 잘 맞고 다음에는 참치 먹으러 가겠습니다',
   '사시미 후기. 특별할것 없는 맛에 살짝 더 비싼가격.조금 더 달라고 했더니 매니저 아주머가 처음에 많이 준거라며 거절. 재방문 필요성 전혀 못느낌.',
   '맛있음',
   '맛있어요♡',
   '좋아요',
   '굿',
   '맛있아요!!',
   '굿',
   '👍',
   '굿',
   '#',
   '#',
   '룸있어서 좋아요',
   '굿',
   '맛있고 푸짐해요',
   '점심에 알탕 먹었는데 알만 잔뜩 들어았어서 좀 질려요 ㅠㅠ',
   '항상 일정한 맛',
   '참치가 정말 맛있어요!']},
 '동원참치 마포롯데캐슬점': {'rank': '4.28',
  'location': '서울 관악구 남부순환로 1766',
  'menu': ['광어', '매운탕', '튀김', '회국수', '

In [27]:
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')
driver.get(url)
driver.implicitly_wait(2)

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_21700\1328651259.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [243]:
  # 검색어 입력
find_search_input('인천 횟집',driver=driver)
driver.implicitly_wait(2)

In [244]:
switch_frame('searchIframe',driver)
driver.implicitly_wait(WAIT)

In [166]:
page_num = 0
pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
if page_num <= 5:
    pages[page_num].click()
    driver.implicitly_wait(WAIT)
else:
    driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
    driver.implicitly_wait(WAIT)
    driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
    driver.implicitly_wait(WAIT)

In [245]:
driver.implicitly_wait(WAIT)
#page_down(40,driver)
sleep(1)
driver.implicitly_wait(1)
#page_down(5,driver)
sleep(2)
driver.execute_script("window.scrollTo(0, 0);")
driver.implicitly_wait(WAIT)
store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
print('total store_list : ', len(store_elements))

total store_list :  10


In [262]:
switch_frame('searchIframe',driver)

In [31]:
with open('naver_울산 횟집_review_dic_page1.json',encoding='utf-8') as f:
    naver_울산_횟집_review_dic_page1 = json.load(f)

In [35]:
len(naver_울산_횟집_review_dic_page1)

34

In [264]:
current_review_dic = dict()
crawling(driver, 9, store_elements,review_dic=current_review_dic)

12
31


In [265]:
current_review_dic

{'자연도횟집': {'rank': '4.26',
  'location': '인천 중구 영종해안남로 32',
  'menu': ['콘치즈',
   '치즈',
   '회무침',
   '우럭',
   '칼국수',
   '개불',
   '가리비',
   '꼬막',
   '라면',
   '산낙지',
   '매운탕',
   '자연산회'],
  'review': ['다시는 가고 싶지 않음. 손님을 대하는 태도의 기본 이 되어 있지 않음. 직원한테 콘치즈 하나더 달랬더 니 알았다고 해서 그런 줄 알았는데 주방에서 ...',
   '애기랑 우럭지리 먹으러 자주 갑니다 회무침이 맛있어요',
   '굿',
   '굿',
   '굿',
   '굿',
   '굿',
   '무심코 들린곳인데 너무 푸짐하고 맛있어요! 사장님\n인심도 너무 좋으시네요~~^^ 칼국수도 맛있었지만 \n무채김치가 어찌나 맛있던지! 영종도 오면 또 ...',
   '좋아요',
   '회 좋아요',
   '굿',
   '포장해서 1kg에 4만원 샀어요 맛도 굿굿',
   '김치가 맛있고 서비스도 좋았어요~\n감사^^',
   '갯벌 뷰에 맛난 회 만족할 만한 스끼다시',
   '맛있어요!',
   '무의도 가는길에 횟집 검색해서 갔어요. 오후4시경 이었는데, 손님은 저희밖에 없어서 전망좋은 2층 자 리에서 편하게 먹었습니다. 굴, 가리비, 산낙지...',
   '사장님친절하시고 회맛있어요',
   '아이 때문에 꽁치도 구워주시고 사장님이 친절하세요!!!',
   '맛있어요',
   '좋아요',
   '스끼다시 좋은데 넘 비싸요',
   '좋아요',
   '좋아요',
   '굳',
   '굿~',
   '그럭저럭',
   '짱맛있었어요 자연산회모듬최고',
   '서비스 굿~~^^',
   '서비스 대박입니다 ㅎㅎㅎ',
   '굿',
   '바다보면서먹기좋아요']}}

In [148]:
current_review_dic

{}

In [149]:
review_dic

{'목포회센타': {'location': '서울 관악구 남부순환로 1766'}}

In [177]:
store_elements[6].click()

In [178]:
switch_frame('entryIframe', driver)

In [179]:
memu_list, review_list = crawling_menu_review(driver)

세꼬시
매운탕
물회
전복
도다리
도다리세꼬시
광어
대구탕
생대구탕
미역국
생우럭탕
모듬회
숭어
숙성회
참치
183
친구들이랑 갔는데 회식하기 좋은 곳 같아요. 다른 텅텅 빈 자리도 많은데 굳이 옆 테이블에 아저씨들이 앉더라고요.. 그래서 진짜 시끄러웠어요. 그게...
생우럭탕 해장에 최고에요~
모임하기 좋은 장소!!
양도 푸짐 나오는음식도 많고 술이술술~~
너무너무 잘먹었습니다 회식으로도 좋아여!!!
회가 무척 맛있었습니다. 서비스는 그냥 그래요
화가 신선해요.
회 좋아하시는분이 가면 천국ㅎㅎㅎ근디 전보다 양이 좀 적어진느낌...
해산물이 신선하고 친절해요
좋아요
사장님 친절하게 서빙해주셨어요. 잘 먹었어요!
다음에 또 방문할게요^^
도다리세꼬시 숭어 광어 등 너무맛있고 친절했어요
메뉴:생대구탕
대구가 적어서 아쉽네요.
맛있게 잘 먹었어요
싱싱한 회와 소주 먹기 좋은 분위기
모임하기 좋아요
soso합니다
맛있어요
좋아요
이용했어요 !
좋아요.
굿
좋은 친구들과 즐거운 시간 보내고 맛나게 잘먹고 왔는데 생각보다 양은 적은듯 싶네요
오래만에 갔었는데…
예전 만큼 양이 많지 않았음 ㅠㅠ
좋아요
굿
깔끔하고 맛있어요!!
맛나요
적극 추천합니다
무난했습니다
굿
음식이 정갈하고 맛있습니다. 그런데 좀 시끄러워요 ㅎ
여기는 믿고 가는 숙성회 맛집입니다. 코로나로 한동
안 못 갔었는데 어제 오랜만에 친구들과의 모임에서 
옛생각도 나고 그 맛을 잃지 않고 유지하고 있...
일행이 늦었는데 따로 음식 잘 챙겨주셨어요. 음식도 맛있고, 잘 먹고 왔습니다.
거리두기가 잘 되어 있어 좋습니다
세꼬시 전문점이라면 활어 횟집 일텐데 사진에 보이 는 회는 특세꼬시모둠에 섞여 나온 것인데 한점을 먹 었는데 상태가 영 아니어서 먹지 않고 남겼습...
도다리세꼬시에 매운탕까지 완벽한 한끼엿어요ㅠㅠㅜ감사히 잘먹었습니다. 재방문의사 있습니당
오랜만에 가도 좋아요 ㅋㅋ 간만에 간건데 여전히 굿~~ㅋㄷ
굿
엄청 친절하십니다 자주 가겠습니다 맛도 좋아요
북창동직장인들점심인기메뉴 생대구탕,지리 특히여
름

In [180]:
memu_list

['세꼬시',
 '매운탕',
 '물회',
 '전복',
 '도다리',
 '도다리세꼬시',
 '광어',
 '대구탕',
 '생대구탕',
 '미역국',
 '생우럭탕',
 '모듬회',
 '숭어',
 '숙성회',
 '참치']

In [181]:
review_list

['친구들이랑 갔는데 회식하기 좋은 곳 같아요. 다른 텅텅 빈 자리도 많은데 굳이 옆 테이블에 아저씨들이 앉더라고요.. 그래서 진짜 시끄러웠어요. 그게...',
 '생우럭탕 해장에 최고에요~',
 '모임하기 좋은 장소!!',
 '양도 푸짐 나오는음식도 많고 술이술술~~',
 '너무너무 잘먹었습니다 회식으로도 좋아여!!!',
 '회가 무척 맛있었습니다. 서비스는 그냥 그래요',
 '화가 신선해요.',
 '회 좋아하시는분이 가면 천국ㅎㅎㅎ근디 전보다 양이 좀 적어진느낌...',
 '해산물이 신선하고 친절해요',
 '좋아요',
 '사장님 친절하게 서빙해주셨어요. 잘 먹었어요!',
 '다음에 또 방문할게요^^',
 '도다리세꼬시 숭어 광어 등 너무맛있고 친절했어요',
 '메뉴:생대구탕\n대구가 적어서 아쉽네요.',
 '맛있게 잘 먹었어요',
 '싱싱한 회와 소주 먹기 좋은 분위기\n모임하기 좋아요',
 'soso합니다',
 '맛있어요',
 '좋아요',
 '이용했어요 !',
 '좋아요.',
 '굿',
 '좋은 친구들과 즐거운 시간 보내고 맛나게 잘먹고 왔는데 생각보다 양은 적은듯 싶네요',
 '오래만에 갔었는데…\n예전 만큼 양이 많지 않았음 ㅠㅠ',
 '좋아요',
 '굿',
 '깔끔하고 맛있어요!!',
 '맛나요',
 '적극 추천합니다',
 '무난했습니다',
 '굿',
 '음식이 정갈하고 맛있습니다. 그런데 좀 시끄러워요 ㅎ',
 '여기는 믿고 가는 숙성회 맛집입니다. 코로나로 한동\n안 못 갔었는데 어제 오랜만에 친구들과의 모임에서 \n옛생각도 나고 그 맛을 잃지 않고 유지하고 있...',
 '일행이 늦었는데 따로 음식 잘 챙겨주셨어요. 음식도 맛있고, 잘 먹고 왔습니다.',
 '거리두기가 잘 되어 있어 좋습니다',
 '세꼬시 전문점이라면 활어 횟집 일텐데 사진에 보이 는 회는 특세꼬시모둠에 섞여 나온 것인데 한점을 먹 었는데 상태가 영 아니어서 먹지 않고 남겼습...',
 '도다리세꼬시에 매운탕까지 완벽한 한끼엿어요ㅠㅠㅜ감사히 잘먹었습니다. 재방문의사 있습니당',

In [156]:
menu_elements  = driver.find_elements(By.CSS_SELECTOR,'div.place_fixed_subtab > div > div:nth-child(1) > div > div > div > span > a > span:first-child')

In [160]:
menu_elements[13].text

'고기'

In [162]:
del menu_elements